# Coursera IBM Capstone Project

# Project Name : Coffee for Sale? 

# Table of Contents

* [Introduction](#introduction)
* [Data](#Data)
* [Analysis](#Analysis)
* [Modelling](#Modelling)
* [Discussion](#Discussion)
* [Conclusion](#Conclusion)

# Introduction <a name="introduction"></a>

## Background

Investors and prospective business owners in today's economy are spoilt for choice in terms of business opportunities. With an abundance of information and a limited timeframe to make decisions, the accuracy of business decisions sometimes boil down to luck or a gut feel. This project aims to assist the decision making process of identifying a lucrative location for an appropriate business investment.  

## Business Problem

Our business problemm answers the question of the which would be the best suited neighborhood to open up a coffee shop in the city of Toronto. This key question will be answered by using the data we scrapped off the wikipedia website and the Foursquare API. Our auidence are potential business owners that are looking to open up a small coffee shop in one of the neighborhoods in Toronto. The location of any business is of paramount importance to any business owner. The location determines the number of potential customer which influences the potential turnover and overall profits made. What we will attempt to do is not easy but will provide a basic and indicative view in terms of identifying possible locations to open a coffee shop.

## Interest

Our interested audience would be investors, currennt and prospective business owners and the general public. Identifying the best hot spots in terms of neighborhoods in an around the Toronto area with attractive even the nosiest of readers of this analysis. 

# Data<a name="Data"></a>

The data we will use will come from:   
* Wikipedia - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M which provides us a list of postal codes, boroughs and neighborhoods within Toronto
* http://cocl.us/Geospatial_data Geo-coordindate data of Toronto
* Foursquare Venues API for understanding the number of venues within the city of Toronto
* Foursquare Likes API for understanding how likes a venues has

Our approch will be the following: 
* Use the wikipedia dataset as a departure point to understand the number of boroughs and neighborhoods within Toronto
* The geo-coordinate data set is keyed by the postal code which will allow us to merge the wikipedia data and the geo-coordinates
* The geo-coordinates will be supplied to the Foursquare API to retrieve the venues within that neighborhood
* The venues will be supplied to the Foursquare Likes API to retrieve the number of likes for that venue

#### Why would we need to know venues?

Understanding the type of venues within a particular neighborhood is of value as our business problem depends on identifying the best location of a specific venue. For areas that have a large number of the same or similar venues would be of interest to our audience as customers would identify the neighborhood as been known for these specific venues. For example, if a neighborhood has a large number of coffee shops, that neighborhood will be known for the coffee shops so if customers are looking for a coffee shop, they would most likely visit that neighborhood. 

#### Why would we need to know how many likes a venue has? 

The number of likes of a venue helps us guage the popularity of neighborhood and venues within it. The idea is that if there are large number of likes within a neighborhood, this neighborhood is a popular neighborhood and the probability of a large customer base is high. This is exactly what our business community will look for. 

#### How are we planning to use this information? 

We will combine the number of venues per category and the likes data into a single data set to be used in a K-Means algorithm. The clustering algorithm will create clusters of possible neighborhoods that we would want to open a coffee shop in.

### Importing Libraries

The following libraries are imported for the analysis and coding. These libraries enable us to fetch data from the web and make API calls. The pandas packages allows us to store and manipulate the retrieved data easily. The folium package enables us to visualise our findings appropriately in a user friendly way. 

In [1]:
import requests
from bs4 import BeautifulSoup
import bs4

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Web Scrapping off Wikipedia

In [2]:
toronto_codes = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
soup = BeautifulSoup(toronto_codes.text, 'lxml')

soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptG7wpAMNAAAUaw2@sAAAES","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"

#### Useful Functions to help work with the data

In [4]:
def column_maker(data=None, position=None):
    """Function to return a list from within a nested list"""
    list_name = []
    for code in data:
        post_code = code[position]
        list_name.append(post_code)
    return list_name

def clean_data(data=None, characters = None):
    """Function to clean out unnecessary characters in a column"""
    list_name = []
    for item in data:
        cleaned = item.rstrip(characters)
        list_name.append(cleaned)
    return list_name

In [5]:
#Retrieving the data from a table within the html document
rows = soup.find(class_="wikitable sortable").find_all('tr')[1:]

#Isolating the values of interest through a list Comprehension
Data = []
for row in rows:
    cell = [i.text for i in row.find_all('td')]
    Data.append(cell)

In [6]:
#Cleaning cells to create a structured dataset
PostalCode   = column_maker(data=Data, position=0)
PostalCode   = clean_data(data=PostalCode, characters='\n')

Borough      = column_maker(data=Data, position=1)
Borough      = clean_data(data=Borough, characters='\n')

Neighborhood = column_maker(data=Data, position=2)
Neighborhood = clean_data(data=Neighborhood, characters='\n')

#Checking structure of the data
print(len(PostalCode), len(Borough), len(Neighborhood))

180 180 180


In [7]:
#Creating DataFrame
Toronto_Data = pd.DataFrame({'PostalCode': PostalCode, 'Borough': Borough, 'Neighborhood': Neighborhood})

Toronto_Data

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [8]:
# Removing the Slashed in the Neighborhood column

Neighbor_Clean = []

for i in Toronto_Data.index:
    neighbor = Toronto_Data.iloc[i, 2]
    replace  = neighbor.replace("/", ",")
    Neighbor_Clean.append(replace)

In [9]:
# Seeing if the cleaning procedure worked
Neighbor_Clean[:10]

['',
 '',
 'Parkwoods',
 'Victoria Village',
 'Regent Park , Harbourfront',
 'Lawrence Manor , Lawrence Heights',
 "Queen's Park , Ontario Provincial Government",
 '',
 'Islington Avenue',
 'Malvern , Rouge']

In [10]:
Toronto_Data['Neighborhood'] = Neighbor_Clean

In [11]:
Toronto_Data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [12]:
len(Toronto_Data.index) == len(Toronto_Data.PostalCode.unique())

True

The above line proves that the number of unique postal codes matches the total number of records in the dataset. 

In [13]:
#Dropping rows with 'Not assigned' in the Borough Column
Toronto_Data.drop(Toronto_Data[Toronto_Data.Borough == 'Not assigned'].index, inplace=True)

print('Number of remaining rows are', len(Toronto_Data.index))
Toronto_Data.set_index('PostalCode', inplace=True)
Toronto_Data.head()

Number of remaining rows are 103


,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park , Harbourfront"
M6A,North York,"Lawrence Manor , Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [14]:
# Check how many 'Not assigned' entries are in the Neighborhood column
len(Toronto_Data.Neighborhood[Toronto_Data.Neighborhood == 'Not assigned'])

0

In [15]:
Toronto_Data.shape

(103, 2)

### Importing Geospatial Data 

The approach used for obtaining the geo spatial data is by importing the file provided and its contents as a data frame. This data frame is merged to the data frame creating from web scrapping the wikipedia site. 

In [16]:
# The code was removed by Watson Studio for sharing.

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Merging data to form a complete data set to use for further analysis

In [17]:
Toronto = Toronto_Data.merge(df_data_0, how='inner', left_index=True, right_index=True)

print(len(Toronto.index))
Toronto.head()

103


,Borough,Neighborhood,Latitude,Longitude
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [18]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=20,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Implementation of the FourSquare API

We will use the Foursquare API to explore the boroughs in the data set. We will have a look at the number of venues in each borough as way to cluster the boroughs accordingly. This technique is very simple but demonstrates the available capability contained with the Foursquare API and the power of geo spatial data. 

In [20]:
# The code was removed by Watson Studio for sharing.

We have created a function to call the Foursquare API for venue data. 

In [21]:
 def nearby_venues(lat=None, lng=None, radius=1000, limit=1000,client_id=None, client_secret=None, version=None):
    # This function takes in the lat - latitude, lng - longitude
    # radius in meters, limit - number of venues returned
    # client id, client secret and version which is obtained from
    # the Foursquare developer console
    # Defining API endpoint for venues
    
    url_four = 'https://api.foursquare.com/v2/venues/explore'
    
    # Defining parameters for the API call
    four_params = {'client_id':client_id, 'client_secret':client_secret,
                   'v':version, 'll':'{},{}'.format(lat, lng), 'radius':radius,'limit':limit}
    # Calling the API with parameters to retrieve the data
    try:
        results = requests.get(url = url_four, params=four_params).json()["response"]['groups'][0]['items']
    except (ConnectionError, KeyError):
        results = requests.get(url = url_four, params=four_params).json()["response"]['groups'][0]['items']
    return results

The venue_data function transform the API output into a more useful format for processing in the form of a data frame. 

In [22]:
 def venue_data(data=None):
# Parameter data must be a json
    id = []
    name = []
    four_lat = []
    four_long= []
    category = []

    for i in data:
        _id   = i['venue']['id']
        _name = i['venue']['name']
        _lat  = i['venue']['location']['lat']
        _lng  = i['venue']['location']['lng']
        _cat  = i['venue']['categories'][0]['name']
        id.append(_id)
        name.append(_name)
        four_lat.append(_lat)
        four_long.append(_lng)
        category.append(_cat)
    
    four_data = pd.DataFrame({'venue_id':id, 'venue':name, 'ven_latitude':four_lat,
                              'ven_longitude':four_long, 'category':category})
    return four_data


The venue_likes function wraps the Foursquare Likes API call for the number of likes per venue. We will use this data to understand the popularity of a specific borough. 

In [23]:
def venue_likes(venue_id=None, client_id=None, client_secret=None, version=None):
    # This function takes in the venue id, client id,
    # client secret and version which can be found
    # on the Foursquare developer console
    likes_api = "https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}".format(
        venue_id,
        client_id,
        client_secret,
        version)
    likes_re = requests.get(likes_api).json()['response']['likes']['count']
    return likes_re

In [24]:
Toronto.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

We now use the function to make a call to the Foursquare API to retrive the venue data. 

In [25]:
# Create container for the retrieved venue data
Toronto_venues = pd.DataFrame([])
# Call foursquare venue API with geo coordinates
for lat, lng, neighborhood, borough in zip(Toronto.Latitude, 
                                           Toronto.Longitude,
                                           Toronto.Neighborhood,
                                           Toronto.Borough):
    # Managing the connection error raised in the Foursquare API 
    ven_data = nearby_venues(lat=lat, lng=lng, client_id=CLIENT_ID,
                                 client_secret=CLIENT_SECRET, version=VERSION)
    # Adding the suburb and city data for the venue 
    _venues = venue_data(data=ven_data)
    _venues['neighborhood'] = neighborhood
    _venues['borough']      = borough
    Toronto_venues = Toronto_venues.append(_venues)


In [28]:
# Create container for the retrieved venue likes data 
ven_likes = []

# Calling the Venue Likes API
for ven in Toronto_venues.venue_id:
    _likes = venue_likes(venue_id=ven, client_id=CLIENT_ID, 
                         client_secret=CLIENT_SECRET,version=VERSION)
    ven_likes.append(_likes)


In [29]:
Toronto_venues['likes'] = ven_likes

In [30]:
Toronto_venues.head()

,venue_id,venue,ven_latitude,ven_longitude,category,neighborhood,borough,likes
0,4b8991cbf964a520814232e3,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,Parkwoods,North York,18
1,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,43.751976,-79.332140,Park,Parkwoods,North York,12
2,57e286f2498e43d84d92d34a,Tim Hortons,43.760668,-79.326368,Café,Parkwoods,North York,7
3,58a8dcaa6119f47b9a94dc05,A&W,43.760643,-79.326865,Fast Food Restaurant,Parkwoods,North York,6
4,4bafa285f964a5203a123ce3,Bruno's valu-mart,43.746143,-79.324630,Grocery Store,Parkwoods,North York,8


In [31]:
Toronto_venues.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4879 entries, 0 to 55
Data columns (total 8 columns):
venue_id         4879 non-null object
venue            4879 non-null object
ven_latitude     4879 non-null float64
ven_longitude    4879 non-null float64
category         4879 non-null object
neighborhood     4879 non-null object
borough          4879 non-null object
likes            4879 non-null int64
dtypes: float64(2), int64(1), object(5)
memory usage: 343.1+ KB


# Analysis

Our analysis starts with creating simple pivots using groupby functions to understand the total number of venues and the aggregate number of likes. This will help us identify possible neighborhoods for our business opportunity or the ideal neighnorhood that we can base our decision on. The data will be wrangled into various tables to help us easily identify the most popular venues in each neighborhood. 

Lets see how many venues there are per Borough in Toronto 

In [32]:
Toronto_venues.groupby('borough').count().sort_values('likes', ascending=False)

,venue_id,venue,ven_latitude,ven_longitude,category,neighborhood,likes
borough,,,,,,,
Downtown Toronto,1671,1671,1671,1671,1671,1671,1671
North York,636,636,636,636,636,636,636
Central Toronto,557,557,557,557,557,557,557
West Toronto,549,549,549,549,549,549,549
East Toronto,403,403,403,403,403,403,403
Scarborough,390,390,390,390,390,390,390
Etobicoke,253,253,253,253,253,253,253
East York,250,250,250,250,250,250,250
York,118,118,118,118,118,118,118


Let's look at how many venues per Neighborhood in Toronto. We have ranked it on the number of likes. 

In [33]:
Toronto_venues.groupby('neighborhood').count().sort_values('likes', ascending=False)

,venue_id,venue,ven_latitude,ven_longitude,category,borough,likes
neighborhood,,,,,,,
Willowdale,112,112,112,112,112,112,112
Studio District,100,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100,100
"Richmond , Adelaide , King",100,100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100,100
"Harbourfront East , Union Station , Toronto Islands",100,100,100,100,100,100,100
"First Canadian Place , Underground city",100,100,100,100,100,100,100
St. James Town,100,100,100,100,100,100,100
Stn A PO Boxes,100,100,100,100,100,100,100


In [34]:
print('There are {} unique categories.'.format(len(Toronto_venues['category'].unique())))

There are 325 unique categories.


Let's look at the most popular Neighborhood based on the aggregate number of likes. 

In [35]:
Toronto_venues.groupby('neighborhood').mean().sort_values('likes', ascending=False)

,ven_latitude,ven_longitude,likes
neighborhood,,,
"Harbourfront East , Union Station , Toronto Islands",43.643054,-79.382748,191.010000
"Toronto Dominion Centre , Design Exchange",43.647645,-79.381551,184.060000
"Richmond , Adelaide , King",43.650182,-79.384009,175.900000
"First Canadian Place , Underground city",43.648105,-79.381791,160.870000
"Commerce Court , Victoria Hotel",43.648436,-79.380054,147.080000
Central Bay Street,43.656753,-79.384282,145.250000
"Garden District, Ryerson",43.656048,-79.379948,142.950000
St. James Town,43.650622,-79.376424,140.660000
Berczy Park,43.647151,-79.376164,119.160000


We will use the One Hot Encoding technique for preprocessing before we use the K-Mean Algorithm

In [36]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['category']], prefix="", prefix_sep="")


In [37]:
Toronto_onehot.insert(0, "borough", Toronto_venues.borough, True)
Toronto_onehot.insert(1, "neighborhood", Toronto_venues.neighborhood, True)
Toronto_onehot.insert(2, "likes", Toronto_venues.likes, True)

In [38]:
Toronto_onehot.head()

,borough,neighborhood,likes,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,

We group the data frame by the 'Borough' and calculate the frequency of a particular venue category

In [39]:
Toronto_grouped = Toronto_onehot.groupby('neighborhood').mean().sort_values('likes', ascending=False)

Toronto_grouped

,likes,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Go

Let's look at the top 5 venues in Toronto before we pass the data through the K-Means algorithm. 

In [40]:
num_top_venues = 5

for borough in Toronto_grouped.index:
    print("----"+borough+"----")
    temp = Toronto_grouped[Toronto_grouped.index == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Harbourfront East , Union Station , Toronto Islands----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.07
2                Hotel  0.06
3                 Park  0.04
4  Japanese Restaurant  0.04


----Toronto Dominion Centre , Design Exchange----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.07
2                 Café  0.07
3  Japanese Restaurant  0.05
4   Seafood Restaurant  0.03


----Richmond , Adelaide , King----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2                Hotel  0.05
3              Theater  0.04
4  Japanese Restaurant  0.03


----First Canadian Place , Underground city----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.07
2                 Café  0.07
3  Japanese Restaurant  0.04
4           Restaurant  0.03


----Commerce Court , Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.10
1    

We remove the likes column and insert it back for the next set of analysis. 

In [41]:
likes_borough = Toronto_grouped[['likes']]

In [42]:
likes_borough

,likes
neighborhood,
"Harbourfront East , Union Station , Toronto Islands",191.010000
"Toronto Dominion Centre , Design Exchange",184.060000
"Richmond , Adelaide , King",175.900000
"First Canadian Place , Underground city",160.870000
"Commerce Court , Victoria Hotel",147.080000
Central Bay Street,145.250000
"Garden District, Ryerson",142.950000
St. James Town,140.660000
Berczy Park,119.160000


In [43]:
Toronto_grouped.drop('likes', axis=1, inplace=True)

The below function groups the data and provides us with a very simple view of the most common venues per neighborhood. 

In [44]:
# Creating a function to format the data
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


We explore the data further and identify the most common venues per Neighborhood in Toronto. 

In [45]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues 
columns = ['neighborhood']
for ind in np.arange(num_top_venues): 
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind])) 
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
    # create a new dataframe
    Toronto_sorted = pd.DataFrame(columns=columns)
    Toronto_sorted['neighborhood'] = Toronto_grouped.index
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :],num_top_venues)


In [46]:
Toronto_sorted = Toronto_sorted.merge(likes_borough, left_on='neighborhood', right_on=likes_borough.index)

In [47]:
Toronto_sorted.head()

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,likes
0,"Harbourfront East , Union Station , Toronto Is...",Coffee Shop,Café,Hotel,Park,Japanese Restaurant,Gym,Theater,Brewery,Pizza Place,Scenic Lookout,191.01
1,"Toronto Dominion Centre , Design Exchange",Coffee Shop,Hotel,Café,Japanese Restaurant,Seafood Restaurant,Theater,Concert Hall,Restaurant,Thai Restaurant,Bookstore,184.06
2,"Richmond , Adelaide , King",Café,Coffee Shop,Hotel,Theater,Tea Room,Restaurant,Japanese Restaurant,Seafood Restaurant,Furniture / Home Store,Sushi Restaurant,175.90
3,"First Canadian Place , Underground city",Coffee Shop,Hotel,Café,Japanese Restaurant,Theater,Restaurant,Seafood Restaurant,Concert Hall,Bookstore,Plaza,160.87
4,"Commerce Court , Victoria Hotel",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Seafood Restaurant,Concert Hall,Bookstore,Thai Restaurant,Theater,147.08


It seems like coffee shops are the order of the day in Toronto!

It is import to note that the neighborhoods with the highest rank of likes and the most common venue being a coffee shop. Harbourfront East, Union Station etc. are the spots of interest. We should look for the cluster that this neighborhood belongs too to identify other possible areas for our investment. 

# Modelling

#### K-Means Clustering

We use the K-Means algorithm for clustering the 'Borough's' within the Toronto city. 5 clusters are chosen for this exerise. 

In [48]:
Toronto_grouped['likes'] = likes_borough.values


In [49]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 3, 3, 3, 3, 3], dtype=int32)

In [50]:
# add clustering labels
Toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [51]:
Toronto_Labelled = Toronto_grouped[['Cluster Labels']]

Toronto_Labelled

,Cluster Labels
neighborhood,
"Harbourfront East , Union Station , Toronto Islands",1
"Toronto Dominion Centre , Design Exchange",1
"Richmond , Adelaide , King",1
"First Canadian Place , Underground city",1
"Commerce Court , Victoria Hotel",3
Central Bay Street,3
"Garden District, Ryerson",3
St. James Town,3
Berczy Park,3


In [52]:
# We create a final data set by merging the clustered data frame to the lat and long coordinates

Toronto_final = Toronto_Labelled.merge(Toronto, left_on=Toronto_Labelled.index, right_on='Neighborhood')

Toronto_final

,Cluster Labels,Borough,Neighborhood,Latitude,Longitude
0,1,Downtown Toronto,"Harbourfront East , Union Station , Toronto Is...",43.640816,-79.381752
1,1,Downtown Toronto,"Toronto Dominion Centre , Design Exchange",43.647177,-79.381576
2,1,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
3,1,Downtown Toronto,"First Canadian Place , Underground city",43.648429,-79.382280
4,3,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648198,-79.379817
5,3,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,3,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
7,3,Downtown Toronto,St. James Town,43.651494,-79.375418
8,3,Downtown Toronto,Berczy Park,43.644771,-79.373306
9,3,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846


In [54]:
Toronto_Venue_Final = Toronto_Labelled.merge(Toronto_sorted, left_on=Toronto_Labelled.index, right_on='neighborhood')

In [55]:
Toronto_Venue_Final

,Cluster Labels,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,likes
0,1,"Harbourfront East , Union Station , Toronto Is...",Coffee Shop,Café,Hotel,Park,Japanese Restaurant,Gym,Theater,Brewery,Pizza Place,Scenic Lookout,191.010000
1,1,"Toronto Dominion Centre , Design Exchange",Coffee Shop,Hotel,Café,Japanese Restaurant,Seafood Restaurant,Theater,Concert Hall,Restaurant,Thai Restaurant,Bookstore,184.060000
2,1,"Richmond , Adelaide , King",Café,Coffee Shop,Hotel,Theater,Tea Room,Restaurant,Japanese Restaurant,Seafood Restaurant,Furniture / Home Store,Sushi Restaurant,175.900000
3,1,"First Canadian Place , Underground city",Coffee Shop,Hotel,Café,Japanese Restaurant,Theater,Restaurant,Seafood Restaurant,Concert Hall,Bookstore,Plaza,160.870000
4,3,"Commerce Court , Victoria Hotel",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Seafood Restaurant,Concert Hall,Bookstore,Thai Restaurant,Theater,147.080000
5,3,Central Bay Street,Coffee Shop,Café,Gastropub,Park,Clothing Store,Sushi Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,145.250000
6,3,"Garden District, Ryerson",Coffee Shop,Gastropub,Japanese Restaurant,Café,Theater,Restaurant,Hotel,Diner,Ramen Restaurant,Plaza,142.950000
7,3,St. James Town,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Hotel,Theater,Italian Restaurant,Plaza,Bakery,140.660000
8,3,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Park,Grocery Store,Bakery,Gym,Beer Bar,119.160000
9,3,Stn A PO Boxes,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Seafood Restaurant,Gastropub,Park,Sporting Goods Shop,Bakery,116.430000


In [62]:
# set color scheme for the clusters
colors = ['green', 'blue', 'red', 'orange', 'gray']

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

We create a map pinned with the neighborhoods and colored accordingly to the clusters that they belong too. 

In [83]:
for lat, lon, neighborhood, borough, cluster in zip(Toronto_final['Latitude'],
                                                    Toronto_final['Longitude'],
                                                    Toronto_final['Neighborhood'],
                                                    Toronto_final['Borough'],
                                                    Toronto_final['Cluster Labels']):
    label = folium.Popup("Neighborhood : " + str(neighborhood) + ", "+ "Borough : " + str(borough)+' Cluster ' + str(cluster), parse_html=True)
    folium.Marker(
        [lat, lon],
        popup=label,
        icon=folium.Icon(color=colors[cluster]),
        fill_color = colors[cluster]).add_to(map_clusters)

map_clusters

We have now clustered and visualised the data we scrapped of wikipedia using the venues we obtained from the Foursquare API. We could use other approaches as well to solve for various business problems that we might want to solve. 

# Discussion

Our discussion will be based on identifying the clusters and the neighborhoods we should be looking at for our business opportunity. 

In [65]:
# We look at the top 5 Most Common Venues of each cluster
common = ['neighborhood','likes', '1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue']

### Cluster 0

Cluster 0 is not the ideal place to open a coffee shop. Looking at the number of likes, these neighborhoods do not seem to be frequented the most. 

In [68]:
cluster_0 = Toronto_Venue_Final.loc[Toronto_Venue_Final['Cluster Labels'] == 0, common].sort_values('likes', ascending=False)

In [69]:
cluster_0.head()

,neighborhood,likes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,"Cliffside , Cliffcrest , Scarborough Village West",18.750000,Pizza Place,Beach,Ice Cream Shop,Burger Joint,Sports Bar
36,Leaside,17.590164,Sporting Goods Shop,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place
37,"Summerhill West , Rathnelly , South Hill , For...",17.144737,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Thai Restaurant
38,"High Park , The Junction South",17.090000,Café,Bar,Coffee Shop,Convenience Store,Thai Restaurant
39,East Toronto,14.467391,Coffee Shop,Café,Greek Restaurant,Ethiopian Restaurant,Fast Food Restaurant


In [70]:
print('There are', len(cluster_0.index), 'neighborhoods in cluster 0')

There are 62 neighborhoods in cluster 0


### Cluster 1

Cluster 1 is of top priority and the neighborhoods within this cluster are the neighborhoods of interest in terms of meeting our requirements for our business problem. This cluster of neighborhoods seem to be known for Coffee Shops and have the most number of likes. This means that customers frequent these areas often and like the venues here. 

In [71]:
cluster_1 = Toronto_Venue_Final.loc[Toronto_Venue_Final['Cluster Labels'] == 1, common].sort_values('likes', ascending=False)

In [72]:
cluster_1.head()

,neighborhood,likes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Harbourfront East , Union Station , Toronto Is...",191.01,Coffee Shop,Café,Hotel,Park,Japanese Restaurant
1,"Toronto Dominion Centre , Design Exchange",184.06,Coffee Shop,Hotel,Café,Japanese Restaurant,Seafood Restaurant
2,"Richmond , Adelaide , King",175.90,Café,Coffee Shop,Hotel,Theater,Tea Room
3,"First Canadian Place , Underground city",160.87,Coffee Shop,Hotel,Café,Japanese Restaurant,Theater


In [73]:
print('There are', len(cluster_1.index), 'neighborhoods in cluster 1')

There are 4 neighborhoods in cluster 1


### Cluster 2

Cluster 2 would be a great location for a Cafe or a Bar. 

In [74]:
cluster_2 = Toronto_Venue_Final.loc[Toronto_Venue_Final['Cluster Labels'] == 2, common].sort_values('likes', ascending=False)

In [75]:
cluster_2.head()

,neighborhood,likes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"Queen's Park , Ontario Provincial Government",95.29,Coffee Shop,Sushi Restaurant,Park,Café,Gastropub
11,"Kensington Market , Chinatown , Grange Park",93.63,Café,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Art Gallery
12,"University of Toronto , Harbord",76.11,Café,Bakery,Bar,Restaurant,Coffee Shop
13,"Little Portugal , Trinity",73.97,Café,Bar,Restaurant,Vegetarian / Vegan Restaurant,Bakery
14,"CN Tower , King and Spadina , Railway Lands , ...",72.00,Harbor / Marina,Café,Coffee Shop,Scenic Lookout,Dog Run


In [76]:
print('There are', len(cluster_2.index), 'neighborhoods in cluster 2')

There are 8 neighborhoods in cluster 2


### Cluster 3

This cluster is second in the order of priority. This cluster is the second most number of likes and the most common venue as a coffee shop. Opening a Coffe Shop in these neighborhoods will be a safe bet. 

In [77]:
cluster_3 = Toronto_Venue_Final.loc[Toronto_Venue_Final['Cluster Labels'] == 3, common].sort_values('likes', ascending=False)

In [78]:
cluster_3.head()

,neighborhood,likes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,"Commerce Court , Victoria Hotel",147.08,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant
5,Central Bay Street,145.25,Coffee Shop,Café,Gastropub,Park,Clothing Store
6,"Garden District, Ryerson",142.95,Coffee Shop,Gastropub,Japanese Restaurant,Café,Theater
7,St. James Town,140.66,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant
8,Berczy Park,119.16,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant


In [79]:
print('There are', len(cluster_3.index), 'neighborhoods in cluster 3')

There are 6 neighborhoods in cluster 3


### Cluster 4

Cluster 4 is of little interest and the seems to be an area for restaurants and Cafe's. 

In [80]:
cluster_4 = Toronto_Venue_Final.loc[Toronto_Venue_Final['Cluster Labels'] == 4, common].sort_values('likes', ascending=False)

In [81]:
cluster_4.head()

,neighborhood,likes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,"Brockton , Parkdale Village , Exhibition Place",49.010000,Café,Coffee Shop,Restaurant,Bar,Furniture / Home Store
19,"St. James Town , Cabbagetown",45.882353,Park,Diner,Japanese Restaurant,Café,Restaurant
20,Christie,34.040000,Korean Restaurant,Café,Coffee Shop,Grocery Store,Mexican Restaurant
21,Davisville North,30.390000,Coffee Shop,Italian Restaurant,Restaurant,Café,Dessert Shop
22,Davisville,30.250000,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café


In [82]:
print('There are', len(cluster_4.index), 'neighborhoods in cluster 4')

There are 17 neighborhoods in cluster 4


# Conclusion

Cluster 1 and 3 are the best locations for our potential investors and business owners. These locations are prime for opening a Coffee Shop and almost gaurantees a reasonable number of customers based on the popularity of these neighborhoods. 

Our analysis is purely based on location data and does not include other data sets such as population density, income bracket of the neighborhoods, average turnover of the venues etc. These additional data sets will probably affect the outcomes but if very difficult to source. The outcome from this analysis coupled with a physical visit to the neighborhood will result in a sound decision being made in terms of identifying the perfect location for the Coffee Shop. 